In [3]:
# prompt: write the code to mount the drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!pip install tf2onnx onnx

In [6]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
import shutil
from pathlib import Path

def ensure_directories():
    """Create necessary directories for model checkpoints and outputs"""
    directories = ['checkpoints', 'models', 'plots']
    for directory in directories:
        os.makedirs(directory, exist_ok=True)
    return directories

def split_dataset(base_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    """
    Split dataset into train, validation, and test sets
    """
    # Create output directories
    splits = ['train', 'validation', 'test']
    for split in splits:
        split_dir = os.path.join(output_dir, split)
        if os.path.exists(split_dir):
            shutil.rmtree(split_dir)
        os.makedirs(split_dir)

    # Process each class
    class_names = []
    for class_folder in os.listdir(base_dir):
        class_path = os.path.join(base_dir, class_folder)
        if not os.path.isdir(class_path):
            continue

        class_names.append(class_folder)

        # Get all images in the class
        images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Shuffle images
        np.random.shuffle(images)

        # Calculate split indices
        n_images = len(images)
        n_train = int(train_ratio * n_images)
        n_val = int(val_ratio * n_images)

        # Split images
        train_images = images[:n_train]
        val_images = images[n_train:n_train + n_val]
        test_images = images[n_train + n_val:]

        # Copy images to respective directories
        for split, split_images in zip(splits, [train_images, val_images, test_images]):
            split_class_dir = os.path.join(output_dir, split, class_folder)
            os.makedirs(split_class_dir, exist_ok=True)

            for img in split_images:
                src = os.path.join(class_path, img)
                dst = os.path.join(split_class_dir, img)
                shutil.copy2(src, dst)

        print(f"Processed {class_folder}: {len(train_images)} train, {len(val_images)} validation, {len(test_images)} test")

    return class_names

def create_data_generators(data_dir, img_height=224, img_width=224, batch_size=32):
    """
    Create data generators for train, validation, and test sets with augmentation
    """
    # Training data generator with augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Validation and test data generators (only rescaling)
    valid_test_datagen = ImageDataGenerator(rescale=1./255)

    # Create generators
    train_generator = train_datagen.flow_from_directory(
        os.path.join(data_dir, 'train'),
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )

    validation_generator = valid_test_datagen.flow_from_directory(
        os.path.join(data_dir, 'validation'),
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    test_generator = valid_test_datagen.flow_from_directory(
        os.path.join(data_dir, 'test'),
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    return train_generator, validation_generator, test_generator



def create_basic_cnn(input_shape, num_classes):
    """Create a basic 3-layer CNN"""
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

def create_vgg_net(input_shape, num_classes):
    """Create VGG16-based model"""
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

def create_mobilenet(input_shape, num_classes):
    """Create MobileNetV2-based model"""
    # Load MobileNetV2 with pretrained ImageNet weights and without the top classifier
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze the base model to prevent its weights from being updated
    base_model.trainable = False

    # Build the full model
    model = Sequential([
        base_model,  # Output shape: (None, 7, 7, 1280)
        GlobalAveragePooling2D(),  # Converts to (None, 1280)
        Dense(128, activation='relu'),  # Hidden layer
        Dropout(0.5),  # Regularization
        Dense(num_classes, activation='softmax')  # Output layer for classification
    ])

    return model

def train_and_evaluate_model(model, train_generator, validation_generator, test_generator, model_name, epochs=50):
    """
    Train and evaluate a model with early stopping and learning rate reduction
    """
    # Define callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=8,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=f'/content/drive/MyDrive/qidk/checkpoints/{model_name}_best.keras',  # Updated extension to .keras
            monitor='val_accuracy',
            save_best_only=True,
            mode='max'
        )
    ]

    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train model
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        callbacks=callbacks
    )

    # Evaluate on test set
    test_results = model.evaluate(test_generator, verbose=1)
    print(f"\nTest results for {model_name}:")
    print(f"Test Loss: {test_results[0]:.4f}")
    print(f"Test Accuracy: {test_results[1]:.4f}")

    return history, test_results

def plot_training_history(history, model_name):
    """Plot and save training history"""
    plt.figure(figsize=(12, 4))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training')
    plt.plot(history.history['val_accuracy'], label='Validation')
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.savefig(f'plots/{model_name}_history.png')
    plt.close()



In [16]:
# Set parameters
BASE_DIR = '/content/drive/MyDrive/qidk/dataset_image_with_pose'  # Replace with your dataset path
OUTPUT_DIR = '/content/drive/MyDrive/qidk/processed_dataset'
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 16

# Create necessary directories
ensure_directories()
os.makedirs('/content/drive/MyDrive/qidk/checkpoints/', exist_ok=True)
# Split dataset
print("Splitting dataset...")
class_names = split_dataset(BASE_DIR, OUTPUT_DIR)
num_classes = len(class_names)

# Create data generators
print("\nCreating data generators...")
train_generator, validation_generator, test_generator = create_data_generators(
    OUTPUT_DIR,
    IMG_HEIGHT,
    IMG_WIDTH,
    BATCH_SIZE
)

input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)
models = {
    # 'basic_cnn': create_basic_cnn(input_shape, num_classes),
    # 'vgg_net': create_vgg_net(input_shape, num_classes),
    'mobile_net': create_mobilenet(input_shape, num_classes)
}

# Train and evaluate models
histories = {}
test_results = {}

for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    history, test_result = train_and_evaluate_model(
        model,
        train_generator,
        validation_generator,
        test_generator,
        model_name
    )

    histories[model_name] = history
    test_results[model_name] = test_result

    os.makedirs('/content/drive/MyDrive/qidk/models', exist_ok=True)

    # Save model
    model.save(f'/content/drive/MyDrive/qidk/models/{model_name}_final.keras', save_format='keras')


print("\nTraining complete! Check the 'plots' directory for training history visualization.")


Splitting dataset...
Processed utkatasana_output: 49 train, 10 validation, 12 test
Processed vrikshasana_output: 37 train, 8 validation, 9 test
Processed natrajasana_output: 49 train, 10 validation, 12 test

Creating data generators...
Found 135 images belonging to 3 classes.
Found 28 images belonging to 3 classes.
Found 33 images belonging to 3 classes.

Training mobile_net...
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 18s 941ms/step - accuracy: 0.4020 - loss: 1.4160 - val_accuracy: 0.5714 - val_loss: 0.6538 - learning_rate: 0.0010
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 203ms/step - accuracy: 0.6503 - loss: 0.7439 - val_accuracy: 0.8929 - val_loss: 0.4870 - learning_rate: 0.0010
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 118ms/step - accuracy: 0.7896 - loss: 0.4698 - val_accuracy: 0.8929 - val_loss: 0.2491 - learning_rate: 0.0010
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 134ms/step - accuracy: 0.8559 - loss: 0.3708 - val_accuracy: 0.9286 - val_loss: 0.2514 - learning_rate: 0.0010
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.9047 - loss: 0.2626 - val_accuracy: 0.9286 - val_loss: 0.1737 - learning_rate: 0.0010
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.8709 - loss: 0.2790 - val_accuracy: 0.9286 - val_loss: 0.1961 - learning_rate: 0.0010
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 113ms/step - accuracy: 0.9417 - loss: 0.1876 - val_accuracy: 0.9286 - va


Test results for mobile_net:
Test Loss: 0.0491
Test Accuracy: 1.0000

Training complete! Check the 'plots' directory for training history visualization.


In [19]:
def convert_to_tflite(model_path, output_path, quantize=True):
    # Load the Keras model
    model = tf.keras.models.load_model(model_path)

    # Create a TFLite converter
    converter = tf.lite.TFLiteConverter.from_keras_model(model)

    if quantize:
        # Apply dynamic range quantization
        converter.optimizations = [tf.lite.Optimize.DEFAULT]

    # Convert the model
    tflite_model = converter.convert()

    # Save the TFLite model
    with open(output_path, 'wb') as f:
        f.write(tflite_model)

    print(f'TFLite model saved to: {output_path}')

# Convert the MobileNet model
convert_to_tflite('/content/drive/MyDrive/qidk/models/mobile_net_final.keras', '/content/drive/MyDrive/qidk/tflite_models/mobile_net.tflite')


ValueError: Layer "dense_6" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, name=keras_tensor_2589>, <KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, name=keras_tensor_2590>]

In [18]:
import tensorflow as tf
import os
import numpy as np

def convert_model_to_tflite(model_path, output_path, quantize=True):
    """
    Convert Keras model to TFLite format with optional quantization

    Args:
        model_path: Path to the saved Keras model
        output_path: Path where to save the TFLite model
        quantize: Whether to apply post-training quantization
    """
    # Load the model
    model = tf.keras.models.load_model(model_path)

    # Create TFLite converter
    converter = tf.lite.TFLiteConverter.from_keras_model(model)

    if quantize:
        # Apply post-training dynamic range quantization
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        # Optionally add more quantization configurations:
        # converter.target_spec.supported_types = [tf.float16]
        # converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

    # Convert the model
    tflite_model = converter.convert()

    # Save the TFLite model
    with open(output_path, 'wb') as f:
        f.write(tflite_model)

    # Get model size
    model_size = os.path.getsize(output_path) / (1024 * 1024)  # Size in MB
    print(f"Model saved to {output_path}")
    print(f"Model size: {model_size:.2f} MB")

    return tflite_model

def generate_metadata(model_name, class_names):
    """
    Generate metadata for the TFLite model

    Args:
        model_name: Name of the model
        class_names: List of class names
    """
    return {
        "name": model_name,
        "description": "Image classification model for pose detection",
        "version": "1.0",
        "author": "Your Name",
        "license": "Apache License 2.0",
        "input_shape": [224, 224, 3],
        "input_dtype": "float32",
        "classes": class_names
    }

def convert_all_models(model_dir, output_dir, class_names):
    """
    Convert all models in the directory to TFLite format

    Args:
        model_dir: Directory containing Keras models
        output_dir: Directory to save TFLite models
        class_names: List of class names
    """
    os.makedirs(output_dir, exist_ok=True)

    # Convert each model
    for model_name in ['basic_cnn', 'vgg_net', 'mobile_net']:
        print(f"\nConverting {model_name}...")

        # Paths
        keras_path = os.path.join(model_dir, f'{model_name}_final.keras')
        tflite_path = os.path.join(output_dir, f'{model_name}.tflite')

        # Convert model
        if os.path.exists(keras_path):
            tflite_model = convert_model_to_tflite(keras_path, tflite_path)

            # Generate and save metadata
            metadata = generate_metadata(model_name, class_names)
            metadata_path = os.path.join(output_dir, f'{model_name}_metadata.json')

            with open(metadata_path, 'w') as f:
                import json
                json.dump(metadata, f, indent=2)

            print(f"Metadata saved to {metadata_path}")
        else:
            print(f"Model {keras_path} not found")

# Usage example
if __name__ == "__main__":
    MODEL_DIR = '/content/drive/MyDrive/qidk/models'
    TFLITE_DIR = '/content/drive/MyDrive/qidk/tflite_models'

    # Convert all models
    convert_all_models(MODEL_DIR, TFLITE_DIR, class_names)


Converting basic_cnn...
Model /content/drive/MyDrive/qidk/models/basic_cnn_final.keras not found

Converting vgg_net...
Model /content/drive/MyDrive/qidk/models/vgg_net_final.keras not found

Converting mobile_net...


ValueError: Layer "dense_6" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, name=keras_tensor_2266>, <KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, name=keras_tensor_2267>]

In [13]:
import tensorflow as tf
import tf2onnx
import os
import logging
from pathlib import Path
import numpy as np

class ModelConverter:
    def __init__(self, input_shape=(224, 224, 3), output_dir='converted_models'):
        self.input_shape = input_shape
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)

        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def create_representative_dataset(self, train_generator):
        """Create a representative dataset for quantization"""
        def representative_dataset():
            for _ in range(100):
                batch = next(train_generator)[0]
                # Normalize the batch if not already done
                if batch.max() > 1:
                    batch = batch / 255.0
                yield [batch.astype(np.float32)]
        return representative_dataset

    def fix_sequential_model(self, model_path):
        """Fix Sequential model for ONNX conversion"""
        model = tf.keras.models.load_model(model_path)
        inputs = tf.keras.Input(shape=self.input_shape, name='input')
        x = inputs
        for layer in model.layers:
            x = layer(x)
        fixed_model = tf.keras.Model(inputs=inputs, outputs=x, name='fixed_model')
        return fixed_model

    def fix_transfer_learning_model(self, model_path):
        """Fix transfer learning models (VGG, MobileNet)"""
        model = tf.keras.models.load_model(model_path)

        # Get the base model layers
        base_model = None
        if 'vgg_net' in str(model_path):
            base_model = tf.keras.applications.VGG16(
                weights='imagenet',
                include_top=False,
                input_shape=self.input_shape
            )
        elif 'mobile_net' in str(model_path):
            base_model = tf.keras.applications.MobileNetV2(
                weights='imagenet',
                include_top=False,
                input_shape=self.input_shape
            )

        base_model.trainable = False

        # Create new model with fixed architecture
        inputs = tf.keras.Input(shape=self.input_shape, name='input')
        x = base_model(inputs)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dense(512, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.5)(x)
        outputs = tf.keras.layers.Dense(model.outputs[0].shape[-1], activation='softmax')(x)

        fixed_model = tf.keras.Model(inputs=inputs, outputs=outputs)

        # Copy weights from the original model's dense layers
        orig_dense_layers = [layer for layer in model.layers if isinstance(layer, tf.keras.layers.Dense)]
        new_dense_layers = [layer for layer in fixed_model.layers if isinstance(layer, tf.keras.layers.Dense)]

        for orig_layer, new_layer in zip(orig_dense_layers, new_dense_layers):
            new_layer.set_weights(orig_layer.get_weights())

        return fixed_model

    def convert_to_tflite(self, model_path, train_generator=None, quantize=True):
        """Convert model to TFLite format"""
        try:
            # Fix model architecture based on type
            if 'vgg_net' in str(model_path) or 'mobile_net' in str(model_path):
                model = self.fix_transfer_learning_model(model_path)
            else:
                model = self.fix_sequential_model(model_path)

            converter = tf.lite.TFLiteConverter.from_keras_model(model)

            if quantize:
                converter.optimizations = [tf.lite.Optimize.DEFAULT]

                if train_generator is not None:
                    converter.representative_dataset = self.create_representative_dataset(train_generator)
                    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
                    converter.inference_input_type = tf.uint8
                    converter.inference_output_type = tf.uint8
                else:
                    converter.target_spec.supported_types = [tf.float16]

            tflite_model = converter.convert()

            model_name = Path(model_path).stem
            quant_type = 'int8' if train_generator else 'float16'
            tflite_path = self.output_dir / f"{model_name}_{quant_type}.tflite"

            with open(tflite_path, 'wb') as f:
                f.write(tflite_model)

            self.logger.info(f"Successfully converted model to TFLite: {tflite_path}")
            return tflite_path

        except Exception as e:
            self.logger.error(f"Error converting to TFLite: {str(e)}")
            raise

    def convert_to_onnx(self, model_path):
        """Convert model to ONNX format"""
        try:
            # Fix model architecture based on type
            if 'vgg_net' in str(model_path) or 'mobile_net' in str(model_path):
                model = self.fix_transfer_learning_model(model_path)
            else:
                model = self.fix_sequential_model(model_path)

            # Set input and output names
            input_signature = (tf.TensorSpec(shape=(None, *self.input_shape),
                                          dtype=tf.float32, name="input"),)

            model_name = Path(model_path).stem
            onnx_path = self.output_dir / f"{model_name}.onnx"

            model_proto, _ = tf2onnx.convert.from_keras(
                model,
                input_signature=input_signature,
                opset=13,
                output_path=str(onnx_path)
            )

            self.logger.info(f"Successfully converted model to ONNX: {onnx_path}")
            return onnx_path

        except Exception as e:
            self.logger.error(f"Error converting to ONNX: {str(e)}")
            raise

def main():
    # Initialize converter
    converter = ModelConverter()

    # Setup data generator for quantization
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/qidk/processed_dataset/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    # Convert models
    model_paths = [
        '/content/models/basic_cnn_final.h5',
        '/content/models/vgg_net_final.h5',
        '/content/models/mobile_net_final.h5'
    ]

    for model_path in model_paths:
        try:
            print(f"\nProcessing {Path(model_path).stem}")

            # Convert to TFLite
            tflite_path = converter.convert_to_tflite(
                model_path,
                train_generator=train_generator,
                quantize=True
            )

            # Convert to ONNX
            onnx_path = converter.convert_to_onnx(model_path)

            print(f"Successfully converted {Path(model_path).stem}")

        except Exception as e:
            logging.error(f"Error processing model {model_path}: {str(e)}")
            continue

if __name__ == "__main__":
    main()

Found 135 images belonging to 3 classes.

Processing basic_cnn_final


Saved artifact at '/tmp/tmp6te5ug2v'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  133503359040176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133502618264400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133503377159120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133503377153664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133502616982224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133502616986096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133502616990320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133502616981168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133502616699120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133502616712320: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
ERROR:__main__:Error converting to ONNX: 'FuncGraph' object has no attribute '_captures'
ERROR:root:Error processing model /content/models/basic_cnn_final.h5: 'FuncGraph' object has no attribute '_captures'



Processing vgg_net_final


ERROR:__main__:Error converting to TFLite: Layer "dense_24" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 7, 7, 512), dtype=float32, sparse=False, name=keras_tensor_1068>, <KerasTensor shape=(None, 7, 7, 512), dtype=float32, sparse=False, name=keras_tensor_1069>]
ERROR:root:Error processing model /content/models/vgg_net_final.h5: Layer "dense_24" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 7, 7, 512), dtype=float32, sparse=False, name=keras_tensor_1068>, <KerasTensor shape=(None, 7, 7, 512), dtype=float32, sparse=False, name=keras_tensor_1069>]



Processing mobile_net_final


ERROR:__main__:Error converting to TFLite: Layer "dense_26" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, name=keras_tensor_1391>, <KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, name=keras_tensor_1392>]
ERROR:root:Error processing model /content/models/mobile_net_final.h5: Layer "dense_26" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, name=keras_tensor_1391>, <KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, name=keras_tensor_1392>]
